# EmoRecSys - Data

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')

Mounted at /content/drive


### Ratings -- for CF

In [ ]:
ratings = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/ratings.csv")
ratings

,id,id_photo,id_survey,like_bool,anger,fear,disgust,sadness,happiness,surprise,neutral,valence,arousal,dominance
0,1,1754,1,1,0,0,0,0,0,0,5,3,1,2
1,2,1785,1,1,0,0,0,2,0,0,0,2,2,3
2,3,123,1,0,0,0,0,4,0,0,0,1,3,1
3,4,2315,1,0,0,0,0,2,0,0,0,2,1,3
4,5,1548,1,0,0,0,0,0,0,0,5,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2441,2546,163,0,0,5,0,5,0,0,0,1,5,1
2441,2442,2097,163,0,0,0,3,0,5,0,0,2,2,1
2442,2443,1898,163,0,0,0,3,5,0,1,0,1,4,1
2443,2444,1871,163,0,0,0,0,5,0,0,0,1,1,1


In [ ]:
# counting rating per photo
n_ratings = ratings.groupby('id_photo').size().reset_index(name='rating_count').sort_values(by='rating_count', ascending=False)
ratings_distribution = n_ratings['rating_count'].value_counts().reset_index(name='num_photos').sort_values(by='rating_count')

ratings_distribution

,rating_count,num_photos
1,1,122
5,3,11
0,5,345
2,6,36
3,7,20
7,10,2
4,11,15
6,12,2


### User Demographics -- for Demo-based

In [ ]:
survey = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/surveys.csv", encoding="utf-8")
survey.columns = ['id_survey', 'age', 'populational_aff', 'gender', 'education', 'city',
       'country_residence', 'date_survey', 'consented', 'hobby_other']
survey

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN
...,...,...,...,...,...,...,...,...,...,...
158,159,41 - 50,White/Caucasian,F,High School,Setúbal,Portugal,2024-06-27 08:39:37,1,NaN
159,160,25 - 30,Asian,F,Bachelor’s Degree,London Borough of Lambeth,United Kingdom,2024-06-27 13:36:50,1,NaN
160,161,31 - 40,White/Caucasian,M,Bachelor’s Degree,Wyoming,United States,2024-06-27 22:06:37,1,Boxing
161,162,25 - 30,Asian,F,Master’s Degree,Overijssel,Netherlands,2024-07-07 14:58:10,1,NaN


#### Hobbies

In [ ]:
hobbies = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/survey_hobbies.csv")
hobbies

,id_survey,id_hobby
0,26,1
1,36,1
2,66,1
3,71,1
4,96,1
...,...,...
371,48,18
372,63,18
373,141,18
374,153,18


In [ ]:
hobbies_grouped = hobbies.groupby('id_survey')['id_hobby'].apply(list).reset_index()
survey = pd.merge(survey, hobbies_grouped, on='id_survey', how='left')

In [ ]:
for hobby in list(set(hobbies['id_hobby'].tolist())):
  survey["hobby_"+str(hobby)] = survey['id_hobby'].apply(lambda x: 1 if isinstance(x, list) and hobby in x else 0)
survey.drop(columns=['id_hobby'], inplace=True)
survey

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other,...,hobby_9,hobby_10,hobby_11,hobby_12,hobby_13,hobby_14,hobby_15,hobby_16,hobby_17,hobby_18
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN,...,0,0,0,1,0,0,0,0,0,0
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN,...,0,0,0,1,0,0,0,0,0,0
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN,...,0,0,0,1,0,0,0,0,0,0
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN,...,0,0,0,1,0,0,0,0,0,0
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,159,41 - 50,White/Caucasian,F,High School,Setúbal,Portugal,2024-06-27 08:39:37,1,NaN,...,0,0,0,0,0,0,0,0,0,0
159,160,25 - 30,Asian,F,Bachelor’s Degree,London Borough of Lambeth,United Kingdom,2024-06-27 13:36:50,1,NaN,...,0,0,0,0,0,1,0,0,0,0
160,161,31 - 40,White/Caucasian,M,Bachelor’s Degree,Wyoming,United States,2024-06-27 22:06:37,1,Boxing,...,0,0,0,0,0,0,0,0,0,0
161,162,25 - 30,Asian,F,Master’s Degree,Overijssel,Netherlands,2024-07-07 14:58:10,1,NaN,...,0,0,0,0,0,1,0,0,0,0


#### Diseases

In [ ]:
diseases = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/survey_diseases.csv")
diseases

,id_survey,id_disease
0,26,1
1,26,2
2,46,2
3,131,2
4,140,2
...,...,...
162,159,11
163,160,11
164,161,11
165,162,11


In [ ]:
diseases_grouped = diseases.groupby('id_survey')['id_disease'].apply(list).reset_index()
survey = pd.merge(survey, diseases_grouped, on='id_survey', how='left')

In [ ]:
for disease in list(set(diseases['id_disease'].tolist())):
  survey["disease_"+str(disease)] = survey['id_disease'].apply(lambda x: 1 if isinstance(x, list) and disease in x else 0)
survey.drop(columns=['id_disease'], inplace=True)
survey

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other,...,hobby_18,disease_1,disease_2,disease_3,disease_4,disease_5,disease_7,disease_9,disease_10,disease_11
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN,...,0,0,0,0,0,0,0,0,0,1
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN,...,0,0,0,0,0,0,0,0,0,1
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN,...,0,0,0,0,0,0,0,0,0,1
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN,...,0,0,0,0,0,0,0,0,0,1
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,159,41 - 50,White/Caucasian,F,High School,Setúbal,Portugal,2024-06-27 08:39:37,1,NaN,...,0,0,0,0,0,0,0,0,0,1
159,160,25 - 30,Asian,F,Bachelor’s Degree,London Borough of Lambeth,United Kingdom,2024-06-27 13:36:50,1,NaN,...,0,0,0,0,0,0,0,0,0,1
160,161,31 - 40,White/Caucasian,M,Bachelor’s Degree,Wyoming,United States,2024-06-27 22:06:37,1,Boxing,...,0,0,0,0,0,0,0,0,0,1
161,162,25 - 30,Asian,F,Master’s Degree,Overijssel,Netherlands,2024-07-07 14:58:10,1,NaN,...,0,0,0,0,0,0,0,0,0,1


#### Visual acuities

In [ ]:
visual_acuities = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/survey_visual_acuities.csv")
visual_acuities

,id_survey,id_visual_acuities
0,1,1
1,3,1
2,13,1
3,16,1
4,22,1
...,...,...
105,14,9
106,104,9
107,92,12
108,148,12


In [ ]:
visual_acuities_grouped = visual_acuities.groupby('id_survey')['id_visual_acuities'].apply(list).reset_index()
survey = pd.merge(survey, visual_acuities_grouped, on='id_survey', how='left')

In [ ]:
for visual_acuity in list(set(visual_acuities['id_visual_acuities'].tolist())):
  survey["visual_acuity"+str(visual_acuity)] = survey['id_visual_acuities'].apply(lambda x: 1 if isinstance(x, list) and visual_acuity in x else 0)
survey.drop(columns=['id_visual_acuities'], inplace=True)
survey

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other,...,disease_10,disease_11,visual_acuity1,visual_acuity2,visual_acuity3,visual_acuity5,visual_acuity6,visual_acuity9,visual_acuity12,visual_acuity20
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN,...,0,1,1,0,0,0,0,0,0,0
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN,...,0,1,0,0,1,0,0,0,0,0
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN,...,0,1,1,0,0,0,0,0,0,0
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN,...,0,1,0,0,0,0,0,0,0,0
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,159,41 - 50,White/Caucasian,F,High School,Setúbal,Portugal,2024-06-27 08:39:37,1,NaN,...,0,1,0,0,0,0,0,0,0,0
159,160,25 - 30,Asian,F,Bachelor’s Degree,London Borough of Lambeth,United Kingdom,2024-06-27 13:36:50,1,NaN,...,0,1,0,0,1,0,0,0,0,0
160,161,31 - 40,White/Caucasian,M,Bachelor’s Degree,Wyoming,United States,2024-06-27 22:06:37,1,Boxing,...,0,1,0,0,0,0,0,0,0,0
161,162,25 - 30,Asian,F,Master’s Degree,Overijssel,Netherlands,2024-07-07 14:58:10,1,NaN,...,0,1,1,0,0,0,0,0,0,0


#### New CSV

In [ ]:
survey.to_csv('/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/demographic.csv', index=False)
survey

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other,...,disease_10,disease_11,visual_acuity1,visual_acuity2,visual_acuity3,visual_acuity5,visual_acuity6,visual_acuity9,visual_acuity12,visual_acuity20
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN,...,0,1,1,0,0,0,0,0,0,0
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN,...,0,1,0,0,1,0,0,0,0,0
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN,...,0,1,1,0,0,0,0,0,0,0
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN,...,0,1,0,0,0,0,0,0,0,0
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,159,41 - 50,White/Caucasian,F,High School,Setúbal,Portugal,2024-06-27 08:39:37,1,NaN,...,0,1,0,0,0,0,0,0,0,0
159,160,25 - 30,Asian,F,Bachelor’s Degree,London Borough of Lambeth,United Kingdom,2024-06-27 13:36:50,1,NaN,...,0,1,0,0,1,0,0,0,0,0
160,161,31 - 40,White/Caucasian,M,Bachelor’s Degree,Wyoming,United States,2024-06-27 22:06:37,1,Boxing,...,0,1,0,0,0,0,0,0,0,0
161,162,25 - 30,Asian,F,Master’s Degree,Overijssel,Netherlands,2024-07-07 14:58:10,1,NaN,...,0,1,1,0,0,0,0,0,0,0


### Photos

In [ ]:
photos = pd.read_csv("/content/drive/MyDrive/THESIS/EmoRecSys/CSVs/photos.csv", sep=";")
photos

,id,file_name,ext,views,id_source,source,height,width
0,1,iaps_mikels_annotated-1019,jpg,0,1019,iaps_mikels_annotated,0,0
1,2,iaps_mikels_annotated-1022,jpg,11,1022,iaps_mikels_annotated,0,0
2,3,iaps_mikels_annotated-1030,jpg,0,1030,iaps_mikels_annotated,0,0
3,4,iaps_mikels_annotated-1040,jpg,0,1040,iaps_mikels_annotated,0,0
4,5,iaps_mikels_annotated-1050,jpg,0,1050,iaps_mikels_annotated,0,0
...,...,...,...,...,...,...,...,...
2880,3079,pexels-photo-19221067,jpeg,0,19221067,pexels,4000,6000
2881,3081,pexels-photo-53442,jpeg,0,53442,pexels,2352,3515
2882,3082,pexels-photo-1636162,jpeg,0,1636162,pexels,3167,4781
2883,3083,pexels-photo-1643166,jpeg,0,1643166,pexels,2986,4508
